In [8]:
import os
import pandas as pd
import numpy as np

In [306]:
path = './all-census-data/'
#read literacy data into dataframe
literacy_data = pd.read_csv(path + 'elementary_2015_16.csv', usecols=list(range(1,9)) + list(range(11,17)))

In [307]:
#convert x females per 1000 males to a decimal (female to 1 male)
# if greater than to avoid running multiple times
if (literacy_data['SEX RATIO'].mean() > 1):
    literacy_data['SEX RATIO'] = literacy_data['SEX RATIO']/1000

In [308]:
#drop rows with null values
literacy_data.dropna(inplace=True)

In [309]:
#group dataframe by 'STATE NAME'
groupedby_state = literacy_data.groupby('STATE NAME')

In [310]:
#aggregate function to call on grouped DataFrame to return weight literacy rates (weighted by respective population size)
def lit_agg(x):
    names = {
        'WEIGHTED MALE LITERACY' : np.average(x['MALE LITERACY'], weights=(literacy_data.loc[x.index, "TOTAL POULATION"]) - (literacy_data.loc[x.index, "SEX RATIO"] * literacy_data.loc[x.index, "TOTAL POULATION"])),
        'WEIGHTED FEMALE LITERACY' : np.average(x['FEMALE LITERACY'], weights=(literacy_data.loc[x.index, "SEX RATIO"] * literacy_data.loc[x.index, "TOTAL POULATION"])),
        'WEIGHTED TOTAL LITERACY' : np.average(x['OVERALL LITERACY'], weights=(literacy_data.loc[x.index, "TOTAL POULATION"])),
        'WEIGHTED SEX RATIO': np.average(x['SEX RATIO'], weights=(literacy_data.loc[x.index, "TOTAL POULATION"])), 
        'WEIGHTED URBAN POPULATION': np.average(x['PERCENTAGE URBAN POPULATION'], weights=(literacy_data.loc[x.index, "TOTAL POULATION"]))
        }
    return pd.Series(names, index=['WEIGHTED MALE LITERACY', 'WEIGHTED FEMALE LITERACY', 'WEIGHTED TOTAL LITERACY', 'WEIGHTED SEX RATIO', 'WEIGHTED URBAN POPULATION'])


In [313]:
#.apply aggregate function
weighted_census_data = groupedby_state.apply(lit_agg)

#bin percentage of urban population in 0-33%, 33-66%, 66-100%
bins = [0, 33, 66, 100]
# weighted_census_data['URBAN POPULATION BIN'] = pd.cut(weighted_census_data['WEIGHTED URBAN POPULATION'], bins)

In [314]:
weighted_census_data

,WEIGHTED MALE LITERACY,WEIGHTED FEMALE LITERACY,WEIGHTED TOTAL LITERACY,WEIGHTED SEX RATIO,WEIGHTED URBAN POPULATION
STATE NAME,,,,,
ANDAMAN AND NICOBAR ISLANDS,89.716853,81.834661,86.247232,0.878864,35.671265
ANDHRA PRADESH,75.359789,60.403824,67.947498,0.996522,29.626491
ARUNACHAL PRADESH,74.029081,59.422039,66.814645,0.922603,12.394013
ASSAM,78.848551,67.135349,73.050808,0.953583,14.080743
BIHAR,72.941194,53.231633,63.733287,0.916073,11.299781
CHANDIGARH,90.540000,81.380000,86.430000,0.818000,97.250000
CHHATTISGARH,85.917936,60.890336,71.382495,0.991189,23.404662
DADRA AND NAGAR HAVELI,86.460000,65.930000,77.650000,0.775000,46.620000
DAMAN AND DIU,91.535172,79.307222,87.052787,0.639507,54.437690


In [121]:
path = './all-census-data/'
 
files = os.listdir(path)

In [6]:
gdp_files = [file for file in files if 'gdp' in file]